# Симетричен блок на трансфер-матрицата

Ще използваме библиотека, която позволява произволно индексиране на масиви, за да можем да работим с 0-базирано броене

In [25]:
if !@isdefined(PACKAGES_INSTALLED)
    using Pkg;
    Pkg.add("OffsetArrays");
    using OffsetArrays;
    PACKAGES_INSTALLED = true;
end

Работим с целочислена аритметика и векторите на състоянията представяме като цели числа, за които състоянието на бит на определена позиция отговоря на спин надолу или нагоре във верижката от спинове за тази позиция.

Така можем да имплементираме операцията транслиране _translation_steps_ - стъпки на верижката от спинове изцяло с побитови операции, без да използваме цикли. Това спестява значително изчислително време.

За целта, нека translation_steps = 2 и работим с 4-битови числа с цел простота на записа:
``` 
~0 <=> 0b1111
0b1111 << 2 = 0b1100
~0b1100 = 0b00011
```
Имаме битова маска, която ни позволява след това с опреация AND да вземем последните два бита на дадено число.

След като имаме нужните битове остава да ги "наместим" на правилните места. Последните _translation_steps_ - бита стават първите, а останалите биват изместени толкова стпъки назад.


In [26]:
function translate(vec, translation_steps, N)
    translation_steps %= N
    head_mask = ~(~0 << translation_steps) # generate the mask that is all zeroes except the last translation_steps bits
    head = vec & head_mask # get the last the last translation_steps of the state vector
    tail = vec >> translation_steps # get the first (n_conf-translation_steps) bits of the vector
    final = (head << (N-translation_steps)) | tail # combine accordingly
    return final
end

translate (generic function with 1 method)

Итеративно тази функция можем да запишем и като:

In [27]:
function translate_loopy(vec, translation_steps, N)
    divisor = 2^(N-1)
    for __ in 1:translation_steps
        v_temp = vec ÷ 2
        vec = v_temp + divisor*(vec-v_temp*2)
    end
    return vec
end

translate_loopy (generic function with 1 method)

Необходима ни е и функция, с която при прилагане на транслационна стъпка да проверяваме дали полученият вектор принадлежи на вече намерен симетричен клас. Тук трябва да стане итеративно, но можем да се възползваме от вградените възможности на Julia за оптимизирано търсене на елемент в обхват, който изпълнява дадено условие. Така получаваме по-компактна и по-бърза функция (binary search)

In [28]:
function find_class(translated_vec, vec)
    res = findfirst(j -> j == translated_vec, 1:vec-1)
    return res === nothing ? -1 : res
end

find_class (generic function with 1 method)

Ако работата в линейно време е приемлива, можем да изпозлваме по-разширен вариант на горната функция:

In [29]:
function find_class_loopy(translated_vec, vec)
    for j in 1:vec-1
        if j == translated_vec
            return j
        end
    end
    return -1
end

find_class_loopy (generic function with 1 method)

Ще са ни необходими и две функции, с които съоветно да добавим първия вектор към новооткрит симетричен клас или да добавим настоящия към вече намерен такъв

In [30]:
function add_new_class(vec, classes, num_vectors_class)
    classes[vec] = vec
    num_vectors_class[vec] = 1
end

add_new_class (generic function with 1 method)

In [31]:
function set_class(vec, class, classes, num_vectors_class)
    classes[vec] = classes[class]
    num_vectors_class[classes[class]] += 1
end

set_class (generic function with 1 method)

Комбинирайки всичко по-горе получаваме следната крайна имплементация за генериране на класовете на симетрия


In [32]:
function classify(n_spins, tr_steps)    
    l = n_spins ÷ tr_steps
    n_conf = 2^n_spins

    classes = OffsetVector(zeros(Int64, n_conf+1), 0:n_conf)
    num_vecs_class = OffsetVector(zeros(Int64, n_conf+1), 0:n_conf)
    n_classes = 2 

    classes[0] = 0
    classes[1] = 1
    num_vecs_class[1] = 1
    num_vecs_class[0] = 1

    new_class_found = false
 
    for vec in 2:n_conf-2
        vec_temp = vec

        for _ in 1:l-1
            vec_temp = translate(vec_temp, tr_steps, n_spins)
            class = find_class(vec_temp, vec)
            if class != -1 # vector belongs to an already found class, update the class
                set_class(vec, class, classes, num_vecs_class)
                new_class_found = false
                break
            else # vector belongs to a new class, continue 
                new_class_found = true
            end
        end
        
        if new_class_found
            add_new_class(vec, classes, num_vecs_class)
            n_classes += 1 
        end

    end 
    
    classes[n_conf-1] = n_conf - 1
    num_vecs_class[n_conf-1] = 1
    n_classes += 1
    
    return (classes, num_vecs_class)
    
end

classify (generic function with 1 method)

Ще добавим към настоящата имплементация и функции за принтиране/записване във файл на блока

In [33]:
function print_classes(classes, num_vecs_class)
    for i in 0:length(classes)-1
        if num_vecs_class[i] > 0
            println(string(num_vecs_class[i]) * "\t" * string(classes[i]))
        end
    end
end

print_classes (generic function with 1 method)

In [34]:
function write_to_file(classes, num_vecs_class, filepath)
    f = open(filepath, "w")
    for i in 0:length(classes)-1
        if num_vecs_class[i] > 0
            write(f, string(num_vecs_class[i]) * "\t" * string(classes[i])*"\n")
        end
    end
    close(f)
end

write_to_file (generic function with 1 method)

In [35]:
classes_t, num_vecs_class_t = classify(4, 1)
print_classes(classes_t, num_vecs_class_t)

1	0
4	1
4	3
2	5
4	7
1	15


In [36]:
n_t = 4
tr_t = 2
classes_t, num_vecs_class_t = classify(n_t, tr_t)
write_to_file(classes_t, num_vecs_class_t, "symblock_julia_n"*string(n_t)*"_tr"*string(tr_t)*".dat")